# Homework 2

<font color='red'>Write your name below:</font>

Shuang Wang 

In this homework, we will train a CNN model on CIFAR-10.
The rest of this notebook contains a template to build a CNN to classify [CIFAR-10](https://www.cs.toronto.edu/%7Ekriz/cifar.html).
The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images. 
Below are several images from CIFAR-10:

![](https://www.tensorflow.org/tutorials/images/cnn_files/output_K3PAELE2eSU9_0.png )

*Your jobs*
1. Read and understand the structure of porvided code.
2. Fill in the missing code block.
3. Tune the hyper-parameters to maximize the accurcy.
4. Execute the whole IPYNB to obtain the results.
    - Export the IPYNB file with results as a HTML.
    - Zip and submit IPYNB and HTML files to Canvas.
    - Missing the output of execution may hurt your grade.

You can find the models with state-of-the art performance on CIFAR-10 [here](https://paperswithcode.com/sota/image-classification-on-cifar-10).

## Hints to Improve Your Results

1. Start from the simple Softmax model.
2. Add Conv and activation layers
3. Tune hyper-parameters, such as batch_size, learning rate, optimizer
4. Try different CNN architectures

First, import the packages or modules required for the competition.

In [1]:
import torch
import torch.nn as nn
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.optim as optim
import torchvision.transforms as transforms

import random

torch.manual_seed(0)
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Define a few hyper parameteers

In [2]:
# feel free to tune the parameters
batch_size = 32
learning_rate = 0.001
epoch = 120

### Loading and normalizing 

Using torchvision, it’s extremely easy to load CIFAR10.

In [3]:
from torch.utils.data import random_split

In [4]:
train_transform = transforms.Compose([
    # Feel free to tune the transform
    transforms.RandomHorizontalFlip(p=.40),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

trainset = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=train_transform)
# split into training and validation set
trainset, valset = random_split(trainset, [42000,8000])
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, 
                                        shuffle=False, num_workers=2)
# load testing set
testset = datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=test_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


## Define the Model


In [5]:
############## Add Your Code Here ##########################

net = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 64 x 16 x 16

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 128 x 8 x 8

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 256 x 4 x 4

            nn.Flatten(), 
            nn.Linear(256*4*4, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 10))

net = net.to(device)

## Define your loss

In [6]:
############## Add Your Code Here ##########################
criterion = nn.CrossEntropyLoss()

## Define your optim

In [7]:
############## Add Your Code Here ##########################
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

## Define the Training Procedure

We will select the model and tune hyper-parameters according to the model's performance on the validation set. 

In [8]:
for epoch in range(epoch):  
    # train the model using train set
    net.train()
    
    num_correct_train = 0
    for i, data in enumerate(trainloader, 0):
        # train your model
        ############################################################
        ############## Add Your Code Here ##########################

        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        out = torch.argmax(outputs.detach(), dim=1)
        num_correct_train += torch.sum(out == labels).item()
        
        
        
        ############################################################

    # print the accuracy of the train batch
    accuracy_train = num_correct_train / len(trainset) * 100
    print(f"Train accuracy at Epoch {epoch+1:3d}:\t\t{accuracy_train:0.4f}%")
        
    # validate the model performance before testing
    net.eval()
    with torch.no_grad():
        num_correct_val = 0
        
        for inputs, labels in valloader:       
        ############################################################
        ############## Add Your Code Here ##########################        
 
            inputs, labels = inputs.to(device), labels.to(device)
            out = net(inputs)

            out = torch.argmax(out, dim=1)
            num_correct_val += torch.sum(out == labels).item()
        ############################################################
        # print the accuracy of the whole validation data
        accuracy_val = num_correct_val / len(valset) * 100
        print(f"Validation accuracy at Epoch {epoch+1:3d}:\t{accuracy_val:0.4f}%")


Train accuracy at Epoch   1:		36.1048%
Validation accuracy at Epoch   1:	47.2375%
Train accuracy at Epoch   2:		52.6905%
Validation accuracy at Epoch   2:	56.1625%
Train accuracy at Epoch   3:		59.1452%
Validation accuracy at Epoch   3:	60.5625%
Train accuracy at Epoch   4:		63.6310%
Validation accuracy at Epoch   4:	63.4750%
Train accuracy at Epoch   5:		66.0357%
Validation accuracy at Epoch   5:	64.6000%
Train accuracy at Epoch   6:		68.3476%
Validation accuracy at Epoch   6:	67.8250%
Train accuracy at Epoch   7:		69.8833%
Validation accuracy at Epoch   7:	68.7500%
Train accuracy at Epoch   8:		71.0571%
Validation accuracy at Epoch   8:	69.3625%
Train accuracy at Epoch   9:		71.8929%
Validation accuracy at Epoch   9:	69.7500%
Train accuracy at Epoch  10:		72.7548%
Validation accuracy at Epoch  10:	71.0625%
Train accuracy at Epoch  11:		73.8429%
Validation accuracy at Epoch  11:	71.1750%
Train accuracy at Epoch  12:		74.6167%
Validation accuracy at Epoch  12:	71.4875%
Train accuracy a

Train accuracy at Epoch 101:		87.1000%
Validation accuracy at Epoch 101:	78.9500%
Train accuracy at Epoch 102:		86.2452%
Validation accuracy at Epoch 102:	77.0625%
Train accuracy at Epoch 103:		86.7000%
Validation accuracy at Epoch 103:	75.5625%
Train accuracy at Epoch 104:		84.3286%
Validation accuracy at Epoch 104:	73.3250%
Train accuracy at Epoch 105:		84.4286%
Validation accuracy at Epoch 105:	72.3000%
Train accuracy at Epoch 106:		86.7905%
Validation accuracy at Epoch 106:	75.8875%
Train accuracy at Epoch 107:		87.4119%
Validation accuracy at Epoch 107:	77.4375%
Train accuracy at Epoch 108:		86.3405%
Validation accuracy at Epoch 108:	76.7875%
Train accuracy at Epoch 109:		87.3048%
Validation accuracy at Epoch 109:	77.7750%
Train accuracy at Epoch 110:		87.7667%
Validation accuracy at Epoch 110:	78.8125%
Train accuracy at Epoch 111:		84.5881%
Validation accuracy at Epoch 111:	77.6000%
Train accuracy at Epoch 112:		87.4714%
Validation accuracy at Epoch 112:	78.3000%
Train accuracy a

## Test your model at the real testing data

In [9]:
net.eval()
num_correct_test = 0
with torch.no_grad():
    for inputs,labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        out = net(inputs)
        out = torch.argmax(out, dim=1)  
        num_correct_test += torch.sum(out==labels).item()
print(f"Test accuracy: {num_correct_test*100/len(testset):0.4f}%")

Test accuracy: 79.6200%


## Grading Critera
1. (10 pts) The submission contains both HTML and IPYNB with the same cell ouputs.
2. (30 pts) The whole IPYNB can be executed without any errors.
3. (10 pts) The CNN model is correctly implemented.
4. (10 pts) The loss function and optimizer are correctly implemeted
5. (20 pts) The training part is correctly implemented.
6. (10 pts) The validation part is correctly implemented.
7. (10 pts) The accuracy is larger than 70% (with regards to the test set) through parameter tuning (learning rate, batch size, optimizer, model structure, transform).